In [ ]:
# default_exp circtools

# Circuit tools

> Implements useful methods for quantum circuits.

Sometimes it is useful to get a list of all the qubits which are "touched" in a circuit. From this we can for example infer the total number of qubits in a circuit without ever specifying them directly. Now, we define a `Circuit` as a list of ticks where each tick is represented by a dictionary and the values of these dictionaries contain sets of qubits. These sets can either contain integers or tuples of two integers of qubit indices. Thus, we need a way to unpack (or flatten) all these tuples first before we can merge the indices together in a list. `unpack` accomplishes this recursively.

In [ ]:
#export
def unpack(seq):
    """Generator for all qubits defined in a (sub)circuit"""
    
    if isinstance(seq, (tuple,set,list)):
        yield from (x for y in seq for x in unpack(y))
    elif isinstance(seq, dict):
        yield from (x for v in seq.values() for y in v for x in unpack(y))
    else:
        yield seq

Let's see how it works. We give it a list of dictionaries as a representation for some circuit.

In [ ]:
circuit = [{'H': {1, 2, 3}}, {'X': {0, 1}, 'CNOT': {(2, 3), (1,5)}}]
print(list(unpack(circuit)))

[1, 2, 3, 0, 1, 2, 3, 1, 5]


As we can see, the output does, exactly as we expected, contains a list of all the qubits in the order we defined them in the circuit where tuples are correctly unpacked as well.

Another useful method which is of particular interest for the creation of multi-parameter noise models is `partition`. This function allows us to scan a `Circuit` for certain gates and return a list of `(tick_index,qubits)`-tuples which represent a certain `partition` of a circuit. For example we could pass the gate symbols for all two-qubit gates to `partition` to retrieve a list of all circuit locations where two-qubit gates are present. This list can be subsequently used for example to randomly select faulty locations with some probability.

In [ ]:
#export
def partition(circuit,gates):
    """Find all (tick,qubit) circuit locations for list of `gates`"""
    
    return [(t_idx,q) for (t_idx,tick) in enumerate(circuit) 
            for g,qs in tick.items() for q in qs if g.upper() in gates]

Let's see what this does for the example circuit we defined above:

In [ ]:
print(partition(circuit, ["X"]))

[(1, 0), (1, 1)]


In [ ]:
print(partition(circuit, ["X","H"]))

[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1)]


In [ ]:
print(partition(circuit, ["CNOT"]))

[(1, (2, 3)), (1, (1, 5))]
